In [1]:
import dates_generation as dg
import os.path
import pandas as pd

Dates generation


In [2]:
def date_generation_process(month, year, holidays_list):
    holidays_list = dg.convert_string_to_date(holidays)
    days_list = dg.weekdays_list(month, year)
    business_days_list = dg.generate_date_from_list(month, year, days_list)
    real_business = dg.extract_holidays_from_weekdays(business_days_list, holidays_list)
    list_of_dates = dg.create_list_of_dates(real_business)
    dictionary = dg.create_dataframe_dictionary(list_of_dates)
    return dictionary

In [3]:
holidays = ["05-05-23", "04-05-23"]

In [4]:
dates_dictionary = date_generation_process(5, 2023, holidays)

In [5]:
dates_dictionary["01-05-2023"]

,articulo,descripcion,chess,wms,fisico


Dataframe base ABC clasification


In [6]:
import dataframe_class_base as dcb

In [7]:
# read ABC base file
fullpath = os.path.abspath(".//abc_mayo_23.xlsx")

df_clasificacion = pd.read_excel(fullpath, sheet_name="Hoja1")

In [8]:
df_abc = dcb.cantidad_segun_abc(df_clasificacion)

In [9]:
df_abc

,articulo,descripcion,clasificacion,cantidad
0,7038,BRAHMA - X12 1000RET#,A,3
1,25668,PEPSI NF PET X6 500CC,A,3
2,7634,BRAHMA - 4X6 LAT 473,A,3
3,25644,7-UP NF PET X6 500CC,A,3
4,7026,QUILMES - X12 1000RET,A,3
...,...,...,...,...
318,78068,STA MAZO DE CARTAS BUDWEISER,C,1
320,78131,"VINO MALBEC MISTERIO X6 0,750 L",C,1
321,78141,TOSTADAS DE ARROZ MOLINOS ALA X24 150GR,C,1
325,78233,YERBA MATE CACHAMATE X12 500GR,C,1


DB Operations


In [10]:
import sqlite_db_generation as sqlgen
import sqlite3
import math
import random

In [11]:
sqlgen.create_db("abc_quantity.db", "abc")

In [12]:
sqlgen.insert_article_df_into_db(df_abc, "abc_quantity.db", "abc")

In [13]:
sqlgen.reset_database_quantity_for_article("abc_quantity.db", "abc")

In [14]:
sqlgen.remove_quant_for_article("abc_quantity.db", "abc",25668)

In [15]:
sqlgen.select_an_article("abc_quantity.db", "abc", 25668)

(25668, 'PEPSI NF PET X6 500CC', 'A', 2)


In [16]:
random_art = sqlgen.select_random_article("abc_quantity.db", "abc", "A")

In [17]:
sqlgen.check_if_article_has_quantity("abc_quantity.db", "abc", random_art[0])

True

In [18]:
def get_max_per_category_per_day(db_name, table_name, dates_dictionary, category):
    cnt = sqlite3.connect(db_name)
    business_days = len(dates_dictionary.keys())
    query = cnt.execute(
        f""" SELECT COUNT(*)
            FROM {table_name}
            WHERE CLASIFICACION = '{category}'; """
    )

    if(category == "A"):
        quantity = query.fetchone()[0] * 3
    elif(category == "B"):
        quantity = query.fetchone()[0] * 2
    else:
        quantity = query.fetchone()[0] * 1

    return math.floor(quantity/business_days)
    

In [19]:
# def add_article_to_dataframe(cod,descrip,dictionary,dictionary_key):
#     if cod in dictionary[dictionary_key].values:
#         return
#     else:
#         df_aux = {'articulo': cod, 'descripcion': descrip, 'chess': "", 'wms': "", 'fisico': "" }
#         dictionary[dictionary_key] = dictionary[dictionary_key].append(df_aux, ignore_index = True)
#     return dictionary

In [20]:
def add_article_to_dataframe(cod,descrip,date_df):
    df_aux = {'articulo': cod, 'descripcion': descrip, 'chess': "", 'wms': "", 'fisico': "" }
    date_df = pd.concat([date_df, pd.DataFrame([df_aux])], ignore_index=True)
    return date_df

In [21]:
def check_article_already_in_df(cod, date_df):
    if cod in date_df.values:
        return True
    else:
        return False

In [22]:
a_articles_per_day = get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"A")
b_articles_per_day = get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"B")
c_articles_per_day = get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"C")


In [23]:
c_articles_per_day

5

In [24]:
def fill_date(date_df, maxA,maxB,maxC,selected_category,max_loop_iter):
    cont_iter = 0
    if(selected_category == "A"):
        contador = 0
        while(contador <= maxA):
            if(sqlgen.check_if_category_has_quantity("abc_quantity.db", "abc", selected_category)):
                rnd_art = sqlgen.select_random_article("abc_quantity.db", "abc", selected_category)
                #print("random: ", rnd_art[0], rnd_art[1], rnd_art[2])
                if(check_article_already_in_df(rnd_art[0],date_df)):
                    #print(rnd_art[0], "already in df")
                    cont_iter+=1
                    if(cont_iter>max_loop_iter):
                        break
                    else:
                        continue
                else:
                    if(sqlgen.check_if_article_has_quantity("abc_quantity.db", "abc", rnd_art[0])):
                        date_df = add_article_to_dataframe(rnd_art[0],rnd_art[1],date_df)
                        sqlgen.remove_quant_for_article("abc_quantity.db", "abc",rnd_art[0])
                        cont_iter=0
                        contador+=1
                    else:
                        continue
            else:
                break
        return date_df
    elif(selected_category == "B"):
        contador = 0
        while(contador <=  maxB):
            if(sqlgen.check_if_category_has_quantity("abc_quantity.db", "abc", selected_category)):
                rnd_art = sqlgen.select_random_article("abc_quantity.db", "abc", selected_category)
                #print("random: ", rnd_art[0], rnd_art[1], rnd_art[2])
                if(check_article_already_in_df(rnd_art[0],date_df)):
                    #print(rnd_art[0], "already in df")
                    cont_iter+=1
                    if(cont_iter>max_loop_iter):
                        break
                    else:
                        continue
                else:
                    if(sqlgen.check_if_article_has_quantity("abc_quantity.db", "abc", rnd_art[0])):
                        date_df = add_article_to_dataframe(rnd_art[0],rnd_art[1],date_df)
                        sqlgen.remove_quant_for_article("abc_quantity.db", "abc",rnd_art[0])
                        contador+=1
                        cont_iter=0
                    else:
                        continue
            else:
                break
        return date_df
    else:
        contador = 0
        while(contador <=  maxC):
            if(sqlgen.check_if_category_has_quantity("abc_quantity.db", "abc", selected_category)):
                rnd_art = sqlgen.select_random_article("abc_quantity.db", "abc", selected_category)
                #print("random: ", rnd_art[0], rnd_art[1], rnd_art[2])
                if(check_article_already_in_df(rnd_art[0],date_df)):
                    #print(rnd_art[0], "already in df")
                    cont_iter+=1
                    if(cont_iter>max_loop_iter):
                        break
                    else:
                        continue
                else:
                    if(sqlgen.check_if_article_has_quantity("abc_quantity.db", "abc", rnd_art[0])):
                        date_df = add_article_to_dataframe(rnd_art[0],rnd_art[1],date_df)
                        sqlgen.remove_quant_for_article("abc_quantity.db", "abc",rnd_art[0])
                        contador+=1
                        cont_iter=0
                    else:
                        continue
            else:
                break
        return date_df        

In [25]:
#reset dates dictionary
dates_dictionary = date_generation_process(5, 2023, holidays)

In [26]:
def check_articles_to_count(df_date,a_articles_per_day,b_articles_per_day,c_articles_per_day):
    articles_per_day = a_articles_per_day+b_articles_per_day+c_articles_per_day
    if(len(df_date)>=articles_per_day):
        return True
    else:
        return False


In [27]:
sqlgen.reset_database_quantity_for_article("abc_quantity.db", "abc")

In [28]:
def random_date_df(dates_dictionary):
    rnd_date = random.choice(list(dates_dictionary.keys()))
    return rnd_date

In [29]:
def Assignation_process():
    while(sqlgen.check_if_item_without_count("abc_quantity.db", "abc")):
        rnd_date=random_date_df(dates_dictionary)
        if(check_articles_to_count(dates_dictionary[rnd_date], a_articles_per_day, b_articles_per_day,c_articles_per_day)):
            continue
        else:
            dates_dictionary[rnd_date] = fill_date(dates_dictionary[rnd_date],a_articles_per_day, b_articles_per_day,c_articles_per_day,"A",10)
            dates_dictionary[rnd_date] = fill_date(dates_dictionary[rnd_date],a_articles_per_day, b_articles_per_day,c_articles_per_day,"B",10)
            dates_dictionary[rnd_date] = fill_date(dates_dictionary[rnd_date],a_articles_per_day, b_articles_per_day,c_articles_per_day,"C",10)



Testing full cycle

In [33]:
dates_dictionary = date_generation_process(5, 2023, holidays)
sqlgen.reset_database_quantity_for_article("abc_quantity.db", "abc")

In [34]:
Assignation_process()

In [35]:
for i in dates_dictionary:
    print(i, len(dates_dictionary[i]))

01-05-2023 14
02-05-2023 14
03-05-2023 14
06-05-2023 0
08-05-2023 10
09-05-2023 14
10-05-2023 14
11-05-2023 14
12-05-2023 14
13-05-2023 14
15-05-2023 14
16-05-2023 14
17-05-2023 14
18-05-2023 14
19-05-2023 15
20-05-2023 14
22-05-2023 14
23-05-2023 14
24-05-2023 11
25-05-2023 14
26-05-2023 14
27-05-2023 14
29-05-2023 13
30-05-2023 14
31-05-2023 14


In [39]:
sqlgen.check_if_item_without_count("abc_quantity.db","abc")

False